In [12]:
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import geocoder
from geopy.geocoders import Nominatim 
from datetime import datetime 

In [13]:
dia_hoy=datetime.today().strftime('%d-%m')
price_today= pd.read_excel(f"df_diario {dia_hoy}.xlsx")
price_today['Latitud'] = price_today['Latitud'].apply(lambda x: x.replace(',','.'))
price_today['Longitud'] = price_today['Longitud'].apply(lambda x: x.replace(',','.'))

In [14]:
price_today.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11654 entries, 0 to 11653
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Provincia                      11654 non-null  object
 1   Municipio                      11654 non-null  object
 2   Localidad                      11654 non-null  object
 3   Código postal                  11654 non-null  int64 
 4   Dirección                      11654 non-null  object
 5   Longitud                       11654 non-null  object
 6   Latitud                        11654 non-null  object
 7   Precio gasolina 95 E5          11654 non-null  object
 8   Precio gasolina 95 E5 Premium  11654 non-null  object
 9   Precio gasolina 98 E5          11654 non-null  object
 10  Precio gasóleo A               11654 non-null  object
 11  Precio gasóleo Premium         11654 non-null  object
 12  Rótulo                         11654 non-null  object
 13  H

In [15]:
geolocator = Nominatim(user_agent="My-App")
location = geolocator.geocode("Avenida Llano Castellano, 51, Madrid") 
latitud_partida=location.latitude
longitud_partida=location.longitude
latitud_partida
longitud_partida


-3.6845524

In [16]:
price_today['Latitud_partida'] = latitud_partida
price_today['Longitud_partida'] = longitud_partida

price_today['Latitud']=price_today['Latitud'].astype('float64')
price_today['Longitud']=price_today['Longitud'].astype('float64')

price_today.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11654 entries, 0 to 11653
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Provincia                      11654 non-null  object 
 1   Municipio                      11654 non-null  object 
 2   Localidad                      11654 non-null  object 
 3   Código postal                  11654 non-null  int64  
 4   Dirección                      11654 non-null  object 
 5   Longitud                       11654 non-null  float64
 6   Latitud                        11654 non-null  float64
 7   Precio gasolina 95 E5          11654 non-null  object 
 8   Precio gasolina 95 E5 Premium  11654 non-null  object 
 9   Precio gasolina 98 E5          11654 non-null  object 
 10  Precio gasóleo A               11654 non-null  object 
 11  Precio gasóleo Premium         11654 non-null  object 
 12  Rótulo                         11654 non-null 

In [17]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)

In [18]:
#si quiero hacer un DF más pequeño en el que aplicar to mercator, hay que crearlo aquí

In [19]:
price_today["mercator_start"] = price_today.apply(lambda x: to_mercator(x['Latitud_partida'],x['Longitud_partida']),axis=1)
price_today["mercator_finish"] = price_today.apply(lambda x: to_mercator(x['Latitud'],x['Longitud']),axis=1)




In [20]:
price_today

,Provincia,Municipio,Localidad,Código postal,Dirección,Longitud,Latitud,Precio gasolina 95 E5,Precio gasolina 95 E5 Premium,Precio gasolina 98 E5,Precio gasóleo A,Precio gasóleo Premium,Rótulo,Horario,Fecha de extracción,Latitud_partida,Longitud_partida,mercator_start,mercator_finish
0,ALBACETE,Abengibre,ABENGIBRE,2250,"AVENIDA CASTILLA LA MANCHA, 26",-1.539167,39.211417,"1,799",No disponible,No disponible,"1,859",No disponible,Nº 10.935,L-D: 07:00-22:00,26-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4364994.97372271 -171359.8982185098)
1,ALBACETE,Alatoz,ALATOZ,2152,"CR CM-332, 46,4",-1.346083,39.100389,"1,849",No disponible,"2,000","1,890","2,000",REPSOL,L-D: 7:00-23:00,26-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4352635.393298916 -149859.0605023663)
2,ALBACETE,Albacete,ALBACETE,2001,"AVENIDA 1º DE MAYO, S/N",-1.868500,38.985667,"1,849",No disponible,"1,949","1,889","1,919",CARREFOUR,L-S: 08:00-22:00; D: 09:00-21:00,26-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4339864.598676129 -208037.3467282484)
3,ALBACETE,Albacete,ALBACETE,2001,CALLE PRINCIPE DE ASTURIAS (POLÍGONO DE ROMICA...,-1.832000,39.054694,"1,839",No disponible,"1,981","1,899","1,989",BP ROMICA,L-D: 06:00-21:30,26-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4347548.649167117 -203972.0657211719)
4,ALBACETE,Albacete,ALBACETE,2001,"CALLE FEDERICO GARCIA LORCA, 5",-1.846361,38.997722,No disponible,No disponible,No disponible,"1,905",No disponible,CEPSA,L-S: 05:00-23:00,26-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4341206.555137643 -205571.5489110633)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11649,ZARAGOZA,Zaragoza,ZARAGOZA,50197,"POLIGONO CALLE TURIASO, 31",-0.974194,41.645722,No disponible,No disponible,No disponible,"1,889",No disponible,NOIL,L-D: 24H,26-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4635980.56675823 -108452.0056893557)
11650,ZARAGOZA,Zaragoza,ZARAGOZA,50197,"RONDA FERIA DE MUESTRAS, DE LA, 20",-0.986389,41.637944,No disponible,No disponible,No disponible,"1,889",No disponible,IDS,L-D: 24H,26-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4635114.72375884 -109809.7456019723)
11651,ZARAGOZA,Zuera,ZUERA,50800,"CARRETERA N-330A KM. 24,000",-0.794000,41.857000,"1,829",No disponible,"1,959","1,859","1,929",REPSOL,L-D: 06:00-22:00,26-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4659499.926134052 -88390.50484689715)
11652,ZARAGOZA,Zuera,ZUERA,50800,"CALLE AUSTRIA, 11,17",-0.782833,41.883583,"1,799",No disponible,No disponible,"1,809","1,849",COOPERATIVA SAN LICER,L-D: 24H,26-04-2022,40.485146,-3.684552,POINT (4506785.804015489 -410445.4916140108),POINT (4662459.13215781 -87147.28239078527)


In [21]:

#calcular distancia
price_today["Distance"] = price_today.apply(lambda x: distance_meters(x['mercator_start'],x['mercator_finish']),axis=1)


In [22]:
price_today.to_excel('price_today.xlsx', index=False)